In [ ]:
!pip install -U efficientnet

In [ ]:
import numpy as np
import pandas as pd 
import os
print(os.listdir("../input"))
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D,concatenate,Concatenate,multiply, LocallyConnected2D, Lambda)
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model
import matplotlib.pyplot as plt
from efficientnet.keras import EfficientNetB3,EfficientNetB4,EfficientNetB5
import skimage.io
from skimage.transform import resize
import imgaug as aug
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from sklearn.utils import class_weight, shuffle
from keras.losses import binary_crossentropy, categorical_crossentropy
#from keras.applications.resnet50 import preprocess_input
from keras.applications.densenet import DenseNet121,DenseNet169,preprocess_input
import keras.backend as K
import tensorflow as tf
from sklearn.metrics import f1_score, fbeta_score, cohen_kappa_score
from keras.utils import Sequence
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import imgaug as ia
import keras.callbacks as callbacks
from keras.callbacks import Callback
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
from skimage.io import MultiImage,imsave,imread
from skimage.transform import resize,rescale
from skimage.color import rgb2gray
from keras.layers import Input,Cropping2D,GlobalAveragePooling2D,Concatenate,Dense,Conv2D
from keras.models import Model,load_model
import keras.applications as kl
from keras.backend import name_scope
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm
import tensorflow as tf
from keras.utils import Sequence
from keras.optimizers import Adam,Adamax
from sklearn.utils import shuffle,class_weight
from keras.utils import to_categorical
import efficientnet.keras as efn

from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose
)

In [ ]:
main_path='../input/prostate-data/'

In [ ]:
image_dim=(224,224,3)
BATCH_SIZE=16
EPOCHS=10

In [ ]:
train_df=pd.read_csv(os.path.join(main_path,'train_data.csv'))
test_df=pd.read_csv(os.path.join(main_path,'test2.csv'))
train_df.head()

In [ ]:
train_df['Gleason Score'].value_counts()

In [ ]:
for i in range(641):
    train_df['file_name'][i] = train_df['file_name'][i].replace('.png', '')
    if train_df['Gleason Score'][i]>5:
        train_df['Gleason Score'][i] = train_df['Gleason Score'][i]-5
    
train_df.head()

In [ ]:
train_df['Gleason Score'].value_counts()

In [ ]:
rows,cols=3,3
fig=plt.figure(figsize=(10,10))
for i in range(1,rows*cols+1):
    img=MultiImage(os.path.join(main_path,'Gleason_train/Gleason_train',train_df.loc[i-1,'file_name']+'.jpg'))
    img = resize(img[-1], (512, 512))
    fig.add_subplot(rows,cols,i)
    plt.imshow(img)
    plt.title('Gleason Score: '+str(train_df.loc[i-1,'Gleason Score']))
plt.show()

In [ ]:
colums=['file_name','Gleason Score']
train_df,val_df=train_test_split(train_df[colums],test_size=0.15)
print('Train shape: {}'.format(train_df.shape))
print('Validation shape: {}'.format(val_df.shape))

In [ ]:
class Generator(Sequence):
    def __init__(self,input_data,batch_size=BATCH_SIZE,dims=image_dim,is_shuffle=True,n_classes=6,is_train=True):
        self.image_ids=input_data[0]
        self.labels=input_data[1]
        self.batch_size=batch_size
        self.dims=image_dim
        self.shuffle=is_shuffle
        self.n_classes=n_classes
        self.is_train=is_train
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(self.image_ids) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.image_ids))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        image_ids_temp = [self.image_ids[k] for k in indexes]
        labels_temp = [self.labels[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(image_ids_temp,labels_temp)

        return X, y
    
    def augment_flips_color(self,p=.5):
        return Compose([
            Flip(),
            RandomRotate90(),
            Transpose(),
            HorizontalFlip(),
            ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.50, rotate_limit=45, p=.75),
            Blur(blur_limit=3),
        ], p=p)
    
    def __data_generation(self, list_IDs_temp,lbls):
        X = np.zeros((self.batch_size, *self.dims))
        y = np.zeros((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            img=MultiImage(os.path.join(main_path,'Gleason_train/Gleason_train',ID+'.jpg'))
            img = resize(img[-1], (self.dims[0], self.dims[1]))
            #Augmentation
            if self.is_train:
                aug = self.augment_flips_color(p=1)
                img = aug(image=img)['image']
                
            X[i] = img

            # Store class
            y[i] = lbls[i]

        return X, to_categorical(y, num_classes=self.n_classes)

In [ ]:
train_gen=Generator([train_df['file_name'].values,train_df['Gleason Score'].values])
val_gen=Generator([val_df['file_name'].values,val_df['Gleason Score'].values],is_shuffle=False,is_train=False)

In [ ]:
class QWKEvaluation(Callback):
    def __init__(self, validation_data=(), batch_size=BATCH_SIZE, interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.batch_size = batch_size
        self.valid_generator, self.y_val = validation_data
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(generator=self.valid_generator,
                                                  steps=np.ceil(float(len(self.y_val)) / float(self.batch_size)),
                                                  workers=1, use_multiprocessing=False,
                                                  verbose=1)
            def flatten(y):
                return np.argmax(y, axis=1).reshape(-1)
            
            score = cohen_kappa_score(self.y_val,
                                      flatten(y_pred),
                                      labels=[0,1,2,3,4,5],
                                      weights='quadratic')
            print("\n epoch: %d - QWK_score: %.6f \n" % (epoch+1, score))
            self.history.append(score)
            if score >= max(self.history):
                print('saving checkpoint: ', score)
                self.model.save('classifier.h5')

In [ ]:
qwk = QWKEvaluation(validation_data=(val_gen, np.asarray(val_df['Gleason Score'][:val_gen.__len__()*BATCH_SIZE])),
                    batch_size=BATCH_SIZE, interval=1)

In [ ]:
in_lay = Input(shape=image_dim)
base_model = EfficientNetB5(weights=None, input_tensor = in_lay, include_top=False)
base_model.load_weights("../input/efficientnet-keras-weights-b0b5/efficientnet-b5_imagenet_1000_notop.h5")

pt_features = base_model(in_lay)
pt_depth = base_model.get_output_shape_at(0)[-1]
bn_features = BatchNormalization()(pt_features)

#%% [markdown]
# ## Attention model

#%%
# here we do an attention mechanism to turn pixels in the GAP on an off
attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', activation = 'relu')(Dropout(0.5)(bn_features))
attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', activation = 'relu')(attn_layer)
attn_layer = Conv2D(1, 
                    kernel_size = (1,1), 
                    padding = 'valid', 
                    activation = 'sigmoid')(attn_layer)
# fan it out to all of the channels
up_c2_w = np.ones((1, 1, 1, pt_depth))
up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
               activation = 'linear', use_bias = False, weights = [up_c2_w])
up_c2.trainable = False
attn_layer = up_c2(attn_layer)

mask_features = multiply([attn_layer, bn_features])
gap_features = GlobalAveragePooling2D()(mask_features)
gap_mask = GlobalAveragePooling2D()(attn_layer)
# to account for missing values from the attention model
gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
gap_dr = Dropout(0.25)(gap)
dr_steps = Dropout(0.25)(Dense(128, activation = 'relu')(gap_dr))
out_layer = Dense(6, activation = 'softmax')(dr_steps)
retina_model = Model(inputs = [in_lay], outputs = [out_layer])

In [ ]:
retina_model.summary()

In [ ]:
from keras.callbacks import (ModelCheckpoint, LearningRateScheduler,
                             EarlyStopping, ReduceLROnPlateau,CSVLogger)

epochs = 10; batch_size = 16
checkpoint = ModelCheckpoint('../working/model_.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, 
                                   verbose=1, mode='auto', epsilon=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=4)
csv_logger = CSVLogger(filename='../working/training_log.csv',
                       separator=',',
                       append=True)

In [ ]:
def kappa_loss(y_true, y_pred, y_pow=2, eps=1e-12, N=5, bsize=32, name='kappa'):
    with tf.name_scope(name):
        y_true = tf.to_float(y_true)
        repeat_op = tf.to_float(tf.tile(tf.reshape(tf.range(0, N), [N, 1]), [1, N]))
        repeat_op_sq = tf.square((repeat_op - tf.transpose(repeat_op)))
        weights = repeat_op_sq / tf.to_float((N - 1) ** 2)
    
        pred_ = y_pred ** y_pow
        try:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [-1, 1]))
        except Exception:
            pred_norm = pred_ / (eps + tf.reshape(tf.reduce_sum(pred_, 1), [bsize, 1]))
    
        hist_rater_a = tf.reduce_sum(pred_norm, 0)
        hist_rater_b = tf.reduce_sum(y_true, 0)
    
        conf_mat = tf.matmul(tf.transpose(pred_norm), y_true)
    
        nom = tf.reduce_sum(weights * conf_mat)
        denom = tf.reduce_sum(weights * tf.matmul(
            tf.reshape(hist_rater_a, [N, 1]), tf.reshape(hist_rater_b, [1, N])) /
                              tf.to_float(bsize))
    
        return nom*0.5 / (denom + eps) + categorical_crossentropy(y_true, y_pred)*0.5


#%%
from keras.callbacks import Callback
class QWKEvaluation(Callback):
    def __init__(self, validation_data=(), batch_size=64, interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.batch_size = batch_size
        self.valid_generator, self.y_val = validation_data
        self.history = []

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict_generator(generator=self.valid_generator,
                                                  steps=np.ceil(float(len(self.y_val)) / float(self.batch_size)),
                                                  workers=1, use_multiprocessing=False,
                                                  verbose=1)
            def flatten(y):
                return np.argmax(y, axis=1).reshape(-1)
            
            score = cohen_kappa_score(flatten(self.y_val),
                                      flatten(y_pred),
                                      labels=[0,1,2,3,4,5],
                                      weights='quadratic')
            print("\n epoch: %d - QWK_score: %.6f \n" % (epoch+1, score))
            self.history.append(score)
            if score >= max(self.history):
                print('saving checkpoint: ', score)
                self.model.save('../working/model_bestqwk.h5')

In [ ]:
qwk = QWKEvaluation(validation_data=(val_gen, np.asarray(val_df['Gleason Score'][:val_gen.__len__()*BATCH_SIZE])),
                    batch_size=batch_size, interval=1)

In [ ]:
for layer in retina_model.layers:
    layer.trainable = True
callbacks_list = [checkpoint, csv_logger, reduceLROnPlat, early, qwk]
retina_model.compile(loss='categorical_crossentropy',
#             loss=kappa_loss,
            optimizer=Adam(lr=1e-4), metrics = ['accuracy'])
retina_model.fit_generator(
    train_gen,
#     steps_per_epoch=np.ceil(float(len(train_x)) / float(batch_size)),
    validation_data=val_gen,
#     validation_steps=np.ceil(float(len(valid_x)) / float(batch_size)),
    epochs=epochs,
    verbose=1,
    workers=1, use_multiprocessing=False,
    callbacks=callbacks_list)